In [5]:
import sys
sys.path.append("../scripts")
import psflib

**WARNING**: LOCAL JWST PRD VERSION PRDOPSSOC-062 CANNOT BE CHECKED AGAINST ONLINE VERSION


In [6]:
psflib?

Type:        module
String form: <module 'psflib' from '/Users/will/Dropbox/globule-seminario/m1-67/psf-notebooks/../scripts/psflib.py'>
File:        ~/Dropbox/globule-seminario/m1-67/scripts/psflib.py
Docstring:   Tools for fitting and removing PSFs from images.